# University Schedule
## Objective 
This notebook has the objective to explore the problem and explain every step to resolve it. After of development of this, I'll implement it in a .py archive for simple execution.
## The problem
The most university in Ecuador has a system to matriculate students which consist of the next Logic:
- Each class has multiple options in days and hours as well as different professors
- Each class has prerequisites for taking it, if a student doesn't approve these requirements can't matriculate in it.
- Each class has a maximum number of students.
- At the start of all semesters or cycles, each student has multiple options for their classes and it is their work to match all classes to create their schedule.
- Creating a schedule isn't all work because if the class is complete and the student couldn't matriculate in it, this schedule may be useless and It's necessary to use another.
The search for the best schedule and some options is a tiring task and It may take several hours and generate much frustration.
## The proposed solution
The solution developed uses the information of each class saved in .xlsx to create different schedule options, and at the moment It can do (It'll be actualized with new features):
- Use the class name, start and end finish hours, class code, and professor name to create different options.
- Explore all possible options which have all classes.
- Export options in .xlsx for the commodity of the user.
- Each option has its self sheet in the file
- Filtration options (In development)

In [1]:
# To take advantage of functionalities from pandas and NumPy, the code will bi focus in use these libraries.
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import datetime as dt

## Problem aboradtion
In this iteration (1) of the solution, it uses brutal force with some considerations for improving the functionality. The idea is to create a matrix whit all options on both axis and check the compatibility between each option of each class, that is verify if the class does not overlap, in this case, the value in the matrix will be true.
After this, I'll create all possible options of schedules and I'll validate them whit the values in the matrix to filter only which can match all classes.

In [2]:
# Read the file and create the first variables to make a compatibility matrix
file = 'example_2.xlsx'
excel_classes = load_workbook("data/"+file)
classes = {}
index1 = []
index2 = []
# Read names of sheets, each sheet represents one class with different options
classes_names = excel_classes.sheetnames
if 'General Observations' in classes_names:
    classes_names.remove('General Observations') # Remove indications in case the user uses the provided layout 
for class_name in classes_names:
    classes[class_name] = pd.read_excel("data/"+file, sheet_name = class_name, index_col = 'Options') # Filling the list with one dataframe for each class
    options = list(classes[class_name].index) # Get the names of the options
    index1 += [class_name]*len(options) # Create index 1 for compatibility matrix
    index2 += options # Create index 2 for compatibility matrix

In [3]:
# This section has some auxiliary functions to divide the problem. I detail a short description of the functionality of each one.

def str2time(hours):
    # str2time() function get str with format %HH:%MM-%HH:%MM and convert it in time variable to return start and end finish hours
    start = dt.time(hour = int(hours.split('-')[0].split(':')[0]), minute = int(hours.split('-')[0].split(':')[-1]))
    finish = dt.time(hour = int(hours.split('-')[-1].split(':')[0]), minute = int(hours.split('-')[-1].split(':')[-1]))
    return start, finish

def create_dataFrame(option):
    # create_dataFrame() function create a dataframe with days and hours of a specific class which are passed as a parameter, days are the indexes, start and finish hours are columns
    days_option = option['Days'].strip().split()
    hours_option = {'Start': [], 'Finish': []}
    for hours in option['Hours'].strip().split():
        start, finish = str2time(hours)
        hours_option['Start'].append(start)
        hours_option['Finish'].append(finish)
    
    return pd.DataFrame({'Days': days_option, **hours_option})

def verify_compatibility(class1, class2):
    # verify_compatibility() function uses two options of different classes to check if they do not overlap. It uses create_dataFrame() function to make one dataframe for each option
    # with the merge() function of pandas I can get only the days when both options have conflict and check if they overlap or not. The function returns true if options do not overlap.
    class1_result = create_dataFrame(class1)
    class2_result = create_dataFrame(class2)    
    class1_result.set_index('Days', inplace = True)
    class2_result.set_index('Days', inplace = True)
    posible_conflicts = pd.merge(class1_result, class2_result, how = 'inner', left_index = True, right_index = True)
    boolean_conflicts = posible_conflicts.apply(lambda x: x['Finish_x'] <= x['Start_y'] or  x['Start_x'] >= x['Finish_y'], axis = 1)
    return  not(False in list(boolean_conflicts))
    #return posible_conflicts

def comb2set(set1, name2, set2, name1 = ''):
    # comb2set() function create a list of dicts whit all possible combinations of two sets, set1 cab be dict or list, in case if it is a list we need name1 for key in result.
    result = []
    for element1 in set1:
        for element2 in set2:
            if name1 != '':
                result.append({name1: element1, name2: element2})
            else:
                result.append({**element1, name2: element2})
    return result

def check1element(keys, values):
    # check1element() function explore compatibility_matrix and return true if the first element does not have conflict whit other options
    boleans = []
    #print(keys, values)
    for i in range(1, len(keys)):
        #print(keys[0], values[0], keys[i], values[i])
        boleans += [compatibility_matrix[keys[0], values[0]].loc[keys[i], values[i]]]
    return not(False in boleans)

def verify_option(schedule):
    # verify_option() use an option of schedule to evaluate if it is possible. It uses check1element() function to explore all possible conflicts, if it finds an overlap, it finishes the search.
    keys = list(schedule.keys())
    values = list(schedule.values())
    validation = check1element(keys, values)
    if len(keys) >= 2 and validation:
        #print('I enter')
        for i in range(1, len(keys)):
            validation = validation and check1element(keys[i:], values[i:])
            #print(keys[i:], values[i:], validation)
            if not validation:
                return validation
            
    return validation

## Create schedule options
The next sections show the process to create all possible options. Two variables are most important in this section: compatibility_matrix, options, and final_options.
- compatibility_matrix has values for compatibility for each option of each class with all other options.
- options has all options without considering if it's possible or not.
- final_options use options and compatibility_matrix to filter only options has all classes.

In [4]:
# Create compatibility_matrix
compatibility_matrix = pd.DataFrame(np.full((len(index1), len(index1)), False), index = [index1, index2], columns = [index1, index2])
classes_comparation = classes_names.copy()
for class_index in classes_names:
    if class_index in classes_comparation:
        classes_comparation.remove(class_index)
    for option_index in classes[class_index].index:
        for class_column in classes_comparation:
            for option_column in classes[class_column].index:
                if class_index != class_column:
                    compatibility_matrix.loc[class_index, option_index][class_column, option_column] = \
                    verify_compatibility(classes[class_index].loc[option_index], classes[class_column].loc[option_column])
                    compatibility_matrix.loc[class_column, option_column][class_index, option_index] = \
                    compatibility_matrix.loc[class_index, option_index][class_column, option_column]
                    #print("I check {} of {} compatibility with {} of {}".format(option_index, class_index, option_column, class_column))
compatibility_matrix

Mass Transfer                \
                                              A      B      C   
Mass Transfer                   A         False  False  False   
                                B         False  False  False   
                                C         False  False  False   
Experimental Design             A          True   True   True   
                                B          True   True   True   
Engineering of Chemical Reactio A          True   True   True   
                                B          True   True   True   
                                C         False   True   True   
Heat Transfer                   A          True  False  False   
                                B         False   True   True   
                                C          True   True   True   
Microbiology                    A          True   True   True   
                                B          True   True   True   
                                C          True   True   True   
                                D          True   True   True   

                                  Experimental Design         \
                                                    A      B   
Mass Transfer                   A                True   True   
                                B                True   True   
                                C                True   True   
Experimental Design             A               False  False   
                                B               False  False   
Engineering of Chemical Reactio A                True   True   
                                B                True   True   
                                C                True   True   
Heat Transfer                   A                True   True   
                                B                True   True   
                                C                True   True   
Microbiology                    A                True   True   
                                B                True   True   
                                C               False   True   
                                D                True  False   

                                  Engineering of Chemical Reactio         \
                                                                A      B   
Mass Transfer                   A                            True   True   
                                B                            True   True   
                                C                            True   True   
Experimental Design             A                            True   True   
                                B                            True   True   
Engineering of Chemical Reactio A                           False  False   
                                B                           False  False   
                                C                           False  False   
Heat Transfer                   A                            True   True   
                                B                            True   True   
                                C                           False   True   
Microbiology                    A                            True  False   
                                B                           False   True   
                                C                            True  False   
                                D                            True   True   

                                         Heat Transfer                \
                                       C             A      B      C   
Mass Transfer                   A  False          True  False   True   
                                B   True         False   True   True   
                                C   True         False   True   True   
Experimental Design             A   True          True   True   True   
                                B   True          True   True   True   
Engineering of Chemical Reactio A  False  

In [5]:
# Create options
options = comb2set(list(classes[classes_names[0]].index), classes_names[1], list(classes[classes_names[1]].index), classes_names[0])
if len(classes_names) > 2:
    for i in range(2, len(classes_names)):
        options = comb2set(options, classes_names[i], list(classes[classes_names[i]].index))
print(len(options))
# Create final_options
final_options = []
for option in options:
    if verify_option(option):
        #print(option)
        final_options.append(option)

216


## Visualization an export
The next section shows the creation of dataframes from each option and their export to the .xlsx file with all important information to give a beautiful result for the user.

In [6]:
def sort_days(day):
    #  sort_days() is used to sort days in the schedules
    days_order = {'Mo': 1, 'Tu': 2, 'We': 3, 'Th': 4, 'Fr': 5, 'Sa': 6, 'Su': 7}
    return days_order[day]

def make_dataframe(schedule):
    # make_dataframe() use an option to create two dataframes: schedule and information
    classes_schedule = []
    data_classes = []
    for keys, values in schedule.items():
        starts = []
        finishes = []
        days = []
        for hours in classes[keys].loc[values, 'Hours'].strip().split():
            start, finish = str2time(hours)
            starts += [start]
            finishes += [finish]
        for day in classes[keys].loc[values, 'Days'].strip().split():
            days += [day]
        for i in range(len(days)):
            class_data = keys + ' ' + values
            classes_schedule.append({days[i]: class_data, 'Start': starts[i], 'Finish': finishes[i]})
            #classes_schedule.append({days[i]: keys, 'Hour': [starts[i], finishes[i]]})
        data_classes.append({'Class': class_data, 'Code': classes[keys].loc[values, 'Code'], 'Professor': classes[keys].loc[values, 'Professor']})
    #return pd.DataFrame(values_result, index)
    option_schedule = pd.DataFrame(classes_schedule).set_index(['Start', 'Finish']).sort_index().fillna('-')  
    columns = list(option_schedule.columns)
    columns.sort(key = sort_days)
    data_classes = pd.DataFrame(data_classes).set_index('Class').fillna('No Data')
    #return option_schedule[columns]
    return option_schedule[columns], data_classes

def save_options(options):
    # save_options() use make_dataframe() function to create a .xlsx file with all options
    for i in range(len(options)):
        schedule, information = make_dataframe(options[i])
        if i == 0:
            with pd.ExcelWriter('results/'+file.split('.')[0]+'_result.'+file.split('.')[-1]) as writer:
                schedule.to_excel(writer, sheet_name = 'Schedule Option '+ str(i+1))
                information.to_excel(writer, sheet_name = 'Schedule Option '+ str(i+1), startcol = 9)
        else:
            with pd.ExcelWriter('results/'+file.split('.')[0]+'_result.'+file.split('.')[-1], mode = 'a', if_sheet_exists = 'overlay') as writer:
                schedule.to_excel(writer, sheet_name = 'Schedule Option '+ str(i+1))
                information.to_excel(writer, sheet_name = 'Schedule Option '+ str(i+1), startcol = 9)
    print('Save Successful')

In [7]:
save_options(final_options)

Save Successful
